In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import tensorflow

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

#config = tf.compat.v1.ConfigProto(device_count={'GPU': 2}, allow_soft_placement=True, log_device_placement=True)
config = tensorflow.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 1.8
tensorflow.compat.v1.Session(config=config)
tensorflow.__version__

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:04:00.0, compute capability: 7.5



'2.4.0-dev20200815'

In [2]:
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import models
import json
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from pathlib import Path
from keras.utils.generic_utils import get_custom_objects
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

from tensorflow.keras.backend import manual_variable_initialization 
manual_variable_initialization(True)

In [5]:
# SPECIFY WHICH EXPERIMENT TO LOAD BY INDEX IN EXPERIMENT-LOG.JSON
ind = 12

In [8]:
from tensorflow.keras.optimizers import Adam, SGD

with open("logs/unvalidated-experiment-log.json") as f:
  experiments = json.load(f)
experiment = experiments['experiments'][ind]
experiment_benchmark = experiments['experiments'][ind]['benchmark']

experiment_benchmark['name'] = experiment_benchmark['benchmark_name']
experiment_benchmark['dataset_folder'] = Path(experiment_benchmark['dataset_folder'])

experiment_benchmark.pop('benchmark_name', None)
experiment_benchmark.pop('num_samples_train', None)
experiment_benchmark.pop('num_samples_validation', None)
experiment_benchmark.pop('num_samples_test', None)

# TODO @Johanna:
# there might be a bug in the way we save the learning rate...
# because we only save the benchmark after the training is done and the
# as_dict function takes the learning rate directly from the optimizer instance,
# this might not be the initial learning rate. 
# Therefore it might be a better way to extract the initial learning rate from the
# learning rate "metric", which was originally intented to visualize learning rate decay 
# and such in tensorboard.

if experiment_benchmark['optimizer'] == 'Adam':
    lr = experiment_benchmark['learning_rate']
    experiment_benchmark['optimizer'] = Adam(learning_rate=lr)
    experiment_benchmark.pop('learning_rate', None)
elif experiment_benchmark['optimizer'] == 'SGD':
    lr = experiment_benchmark['learning_rate']
    experiment_benchmark['optimizer'] = SGD(learning_rate=lr)
    experiment_benchmark.pop('learning_rate', None)
else:
    raise NotImplementedError()

posw = tensorflow.constant(experiment_benchmark['positive_weights'], dtype=tensorflow.float32)
negw = tensorflow.constant(experiment_benchmark['negative_weights'], dtype=tensorflow.float32)

experiment_benchmark.pop('positive_weights', None)
experiment_benchmark.pop('negative_weights', None)

experiment_benchmark['test_labels'] = "test.csv"

# TODO @Johanna:
# when training our model with the experiment_exec.py we also provide a preprocessing function from keras for the specific model.
# e.g. preprocess_input_fn = tf.keras.applications.densenet.preprocess_input for densenet
# see line 171 of experiment_exec.py
fixme()


benchmark = Benchmark(**experiment_benchmark)
testgen = benchmark.testgen
traingen = benchmark.traingen
valgen = benchmark.valgen

print(experiment_benchmark['name'])

Chexpert_BCE_E3_B32_C0_N12_D256_DS9505_2LR1_LF5_SGD


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from src.metrics.metrics import SingleClassMetric, NaNWrapper, F2Score, FBetaScore

# TODO @Johanna:
# have a look at the load_model notebook in src/architecture/simple
# that approach works flawlessly
# Idea: Instead of load_model just rebuilt the model yourself using SimpleBaseArchitecture 
# or even better directly with simple_architecture_experiment  and then use load_weights.
# That way we can also load the weights after any given epoch and not just when experiment.save was called.
# But from my understanding these weight files have the same h5 (aka hdf5) format as the export from model.save()

path = os.path.join("models/", experiment['name'], experiment['filename'])
reconstructed_model = models.load_model('models/DenseNet121_Chexpert_BCE_E3_B32_C0_N12_D256_DS9505_2LR1_LF5_SGD/DenseNet121_Chexpert_BCE_E3_B32_C0_N12_D256_DS9505_2LR1_LF5_SGD_20200820-085707', custom_objects={'NanWrapper':NaNWrapper, 'SingleClassMetric':SingleClassMetric, 'F2Score':F2Score, 'FBetaScore':FBetaScore}, compile=False)

if experiment_benchmark['loss'] == "weighted_binary_crossentropy":
    loss = WeightedBinaryCrossentropy(posw, negw)
elif experiment_benchmark['loss'] == "binary_crossentropy":
    loss = BinaryCrossentropy()
else:
    raise NotImplementedError()

In [11]:
# testgen.on_epoch_end()
print(len(testgen))

234


In [12]:
thresholds = [0.3, 0.4, 0.5]

In [ ]:
compute_class_weight(traingen)

In [13]:
groundtruth_label = testgen.get_labels_nonan()
predictions = reconstructed_model.predict(testgen, steps=len(testgen), verbose=1)

for t in thresholds:
    print('current threshold: ', t)
    predictions_bool = (predictions >= t)
    y_pred = np.array(predictions_bool, dtype=int)

    # report = classification_report(groundtruth_label, y_pred, target_names=list(CHEXPERT_BENCHMARKS['WBCE_E5_B32_C0'].label_columns))
    # print('sklearn report: ', report)
    
    pre = Precision(thresholds=t)
    pre.update_state(groundtruth_label, predictions_bool)
    print('TF Precision: ', pre.result())
    pre.reset_states()
    
    rec = Recall(thresholds=t)
    rec.update_state(groundtruth_label, predictions_bool)
    print('TF Recall: ', rec.result())
    rec.reset_states()
    


234/234 [==============================] - 8s 35ms/step
current threshold:  0.3
TF Precision:  tf.Tensor(0.53846157, shape=(), dtype=float32)
TF Recall:  tf.Tensor(0.23076923, shape=(), dtype=float32)
current threshold:  0.4
TF Precision:  tf.Tensor(0.53846157, shape=(), dtype=float32)
TF Recall:  tf.Tensor(0.23076923, shape=(), dtype=float32)
current threshold:  0.5
TF Precision:  tf.Tensor(0.53846157, shape=(), dtype=float32)
TF Recall:  tf.Tensor(0.23076923, shape=(), dtype=float32)


In [50]:
from src.datasets.generator import ImageDataGenerator

test_labels = pd.read_csv("data/chexpert/full/test.csv")
dataset_folder = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY"))
label_columns = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']
testgen = ImageDataGenerator(test_labels,
                                          dataset_folder,
                                          label_columns, batch_size=1)

In [51]:
preds = reconstructed_model.predict(testgen, steps=len(testgen), verbose=1)
preds

234/234 [==============================] - 5s 20ms/step


array([[7.0323948e-11, 1.3200820e-13, 6.3784039e-01, ..., 9.9872369e-01,
        8.8423330e-10, 5.4291327e-09],
       [6.9098852e-11, 3.2104817e-13, 8.9581883e-01, ..., 9.8847753e-01,
        4.1464627e-09, 7.9451556e-09],
       [1.4373948e-09, 7.8965681e-13, 3.1074613e-01, ..., 9.9728429e-01,
        2.6169080e-09, 1.0400851e-08],
       ...,
       [8.1647411e-11, 3.1016143e-13, 8.5965234e-01, ..., 9.9179405e-01,
        1.8934747e-09, 7.4422180e-09],
       [1.1734806e-09, 1.4159610e-12, 3.2562828e-01, ..., 9.7326916e-01,
        1.4679187e-09, 1.5118714e-07],
       [1.0182928e-11, 1.4958024e-14, 6.7234731e-01, ..., 9.9651295e-01,
        2.6680114e-10, 2.4801730e-09]], dtype=float32)

In [53]:
bools = preds > 0.5
for x in bools:
    print(x)

[False False  True False False False False False False  True False False]
[False False  True False False False False False False  True False False]
[False False False False False False False False False  True False False]
[False False False False False False False False False  True False False]
[False False False False False False False False False  True False False]
[False False  True False False False False False False  True False False]
[False False  True False False False False False False  True False False]
[False False  True False False False False False False  True False False]
[False False  True False False False False False False  True False False]
[False False  True False False False False False False  True False False]
[False False  True False False False False False False  True False False]
[False False False False False False False False False  True False False]
[False False  True False False False False False False  True False False]
[False False  True False False False F

In [15]:
report = classification_report(groundtruth_label, y_pred, target_names=benchmark.label_columns)
print('sklearn report: ', report)

sklearn report:                              precision    recall  f1-score   support

Enlarged Cardiomediastinum       0.00      0.00      0.00       109
              Cardiomegaly       0.00      0.00      0.00        68
              Lung Opacity       0.54      1.00      0.70       126
               Lung Lesion       0.00      0.00      0.00         1
                     Edema       0.00      0.00      0.00        45
             Consolidation       0.00      0.00      0.00        33
                 Pneumonia       0.00      0.00      0.00         8
               Atelectasis       0.00      0.00      0.00        80
              Pneumothorax       0.00      0.00      0.00         8
          Pleural Effusion       0.00      0.00      0.00        67
             Pleural Other       0.00      0.00      0.00         1
                  Fracture       0.00      0.00      0.00         0

                 micro avg       0.54      0.23      0.32       546
                 macro avg   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
eval_res = reconstructed_model.evaluate(
            x=testgen, steps=len(testgen), verbose=1)

In [ ]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

#config = tf.compat.v1.ConfigProto(device_count={'GPU': 2}, allow_soft_placement=True, log_device_placement=True)
config = tensorflow.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 1.8
tensorflow.compat.v1.Session(config=config)
tensorflow.__version__

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

columns_5 =  ['Cardiomegaly',
                'Edema',
                'Consolidation',
                'Atelectasis',
                'Pleural Effusion']

chexpert_benchmarks, _ = generate_benchmarks(path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                                     name_suffix="_D256_DS0595_savetest7",
                                                     classes=columns_5,
                                                     # test_labels = "test.csv",
                                                     split_valid_size = 0.2,
                                                     split_test_size = 0.2,
                                                     train_labels = "train.csv",
                                                     # nan_replacement = -1, #float("NaN"),
                                                     batch_sizes = {"b": 32},
                                                     epoch_sizes = {"e": 1},
                                                     dim=(256, 256),
                                                     crop = {"C0": False},
                                                     split_seed = 6122156,
                                                     augmentation=None)#"affine")
testgen = chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].testgen
label_columns = chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].label_columns
del chexpert_benchmarks

In [ ]:
import tensorflow
model = tensorflow.keras.models.load_model('models/DenseNet121_Chexpert_BCE_E1_B32_C0_N5_D256_DS0595_savetest7/DenseNet121_Chexpert_BCE_E1_B32_C0_N5_D256_DS0595_savetest7_20200819-125620.h5', compile=False)
predstwo = model.predict(testgen, steps=len(testgen), verbose=1)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report
groundtruth_label = testgen.get_labels_nonan()
predictions_bool = (predstwo >= 0.5)
y_pred = np.array(predictions_bool, dtype=int)
report = classification_report(groundtruth_label, y_pred, target_names=label_columns)
print('sklearn report: ', report)

I think something doesnt work with the masked loss function, I get really low values here. Fix.